In [9]:
print("OK")

OK


In [10]:
%pwd

'c:\\Users\\HP\\Projects\\GenerativeAI\\Medical-Chat-Bot\\research'

In [11]:
import os

In [12]:
os.chdir("../")

In [13]:
# %pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
# from langchain_community.embeddings import HuggingFaceEmbeddings

In [61]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [16]:
extracted_data = load_pdf_files("Data")

In [17]:
# extracted_data

In [18]:
len(extracted_data)

759

In [19]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [20]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks :", len(text_chunks))

Length of Text Chunks : 6973


In [21]:
# text_chunks

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\HP\AppData\Local\Temp\ipykernel_22376\30168171.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\HP\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# result_query = embedding.embed_query('Hello World')
# print('Length of Result Query:', len(result_query))

In [24]:
# result_query

In [ ]:
from dotenv import load_dotenv
# import os
load_dotenv()

True

In [46]:
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [47]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)

In [48]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [49]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [62]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from .env
load_dotenv()

# Only keep GOOGLE_API_KEY since you are using Gemini
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Missing GOOGLE_API_KEY. Please add it to your .env file")

# Set it for langchain_google_genai
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash")


In [31]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name=index_name
)

KeyboardInterrupt: 

In [32]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [33]:
docsearch

# Add more data to the existing Pinecone index

In [34]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [35]:
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001E2D86F37F0>, search_kwargs={'k': 3})

In [50]:
retrieved_docs = retriever.invoke("What is tuberculosis?")

In [51]:
retrieved_docs

[Document(id='c4c64843-88cd-4160-ae5b-17dfc3807b48', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 728.0, 'page_label': '729', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\Medical_book.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='may be found in the patient’s stools, vomit, sputum, or\nskin lumps (for lung flukes). In the case of lung flukes, it\nis important for the doctor to rule out tuberculosis as a\npossible diagnosis. A tuberculosis skin test and chest x\nray will usually be sufficient to do this.\nGALE ENCYCLOPEDIA OF MEDICINE 2\n1343\nFluke infections'),
 Document(id='ea9f589a-ef4a-4dc1-b628-4016d2d97821', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 136.0, 'page_label': '137', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\Medical_book.

In [ ]:
# from langchain_openai import ChatOpenAI

# chatModel = ChatOpenAI(model="gpt-4o")

In [ ]:
# pip install langchain-google-genai


   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 989.2 kB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 960.2 kB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 960.2 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 968.5 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 968.5 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 968.5 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 968.5 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 968.5 kB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 573.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   -------------------

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash")


In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [40]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "I can't help with that. Use three sentences maximum and keep the "
    "answer concise. If you think that that is a more relevant context, "
    "feel free to include it."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [63]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# question_answer_chain = create_stuff_documents_chain(OpenAI, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [64]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

I am sorry, but this document does not contain the answer to your question.


In [55]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

I am sorry, but the context provided does not contain information about acne. Therefore, I can't help you with that.


In [56]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

I am sorry, but the document does not contain information on the treatment of acne. The document discusses the treatment of contact dermatitis, which includes eliminating or avoiding the source of irritation and using corticosteroid creams to lessen inflammation and relieve irritation. Contact dermatitis that leads to a bacterial skin infection is treated with antibiotics.


In [65]:
response = rag_chain.invoke({"input": "what is tuberculosis ?"})
print(response["answer"])

Tuberculosis is an infectious disease that typically affects the lungs, but it can also affect other areas of the body. Common symptoms include fever, weight loss, and coughing up blood. In the case of lung flukes, it is important for the doctor to rule out tuberculosis as a possible diagnosis.
